---
원본

In [20]:
# import sys
# from PyQt5.QtWidgets import *
# from enum import Enum
# import win32com.client
 
# # 설명: 주식 한 종목의 매수/정정/취소 주문 처리 및 실시간 시세와 주문 체결 처리 예제
# #   매수주문 - 현재가/10차 호가를 구해 10차 호가로 매수 주문 냄
# #   정정주문 - 누를 때 마다 호가를 9차 > 8차 > 7차 식으로 올려 정정주문 (가격은 실시간으로 업데이트 된 가격임)
# #   취소주문 - 취소 주문
 
# # CpEvent: 실시간 현재가 수신 클래스 - 아래 3가지 실시간 시세 수신
# #       실시간 체결 현재가
# #       실시간 10차 호가
# #       실시간 주문 체결
# # CpPBStockCur : (실시간)현재가 체결 요청 클래스
# # CpPBStockBid : (실시간)현재가 10차 호가 요청 클래스
# # CpPBConclusion : (실시간)주문체결 데이터 요청 클래스
# # CpRPOrder : (RQ/RP)주식 매수/매도/정정 통신 클래스
# # CpRPCurrentPrice : (RQ/RP)주식 현재가 통신 클래스
# # OrderMain : 주문/체결에 대한 핵심 처리 클래스
# #       매수/정정/취소 주문 버튼 클릭에 대한 이벤트 처리
# #       실시간 주문 체결 업데이트에 따른 주문 상태 업데이트
 
 
# # enum 주문 상태 세팅용
# class orderStatus(Enum):
#     nothing = 1          # 별 일 없는 상태
#     newOrder = 2          # 신규 주문 낸 상태
#     orderConfirm = 3      # 신규 주문 처리 확인
#     modifyOrder = 4     # 정정 주문 낸 상태
#     cancelOrder = 5      # 취소 주문 낸 상태
 
# # 현재가와 10차 호가를 저장하기 위한 단순 저장소
# class stockPricedData:
#     def __init__(self):
#         self.cur = 0        # 현재가
#         self.offer = []     # 매도호가
#         self.bid = []       # 매수호가
 
# # 주문 체결 pb 기록용(종료 시 받은 데이터 print)
# class orderHistoryData:
#     def __init__(self):
#         self.flag = "123"
#         self.code = "123"
#         self.price = 0
#         self.orderamount = 0
#         self.contamount = 0
#         self.etc = "123"
 
#     def sethistory(self, flag, code, price, amount, contamount, ordernum, etc):
#         self.flag = flag
#         self.code = code
#         self.price = price
#         self.orderamount = amount
#         self.contamount = contamount
#         self.ordernum = ordernum
#         self.etc = etc
 
#     def printhistory(self):
#         print(self.flag, self.code, "가격:", self.price, "수량:", self.orderamount, "체결수량:", self.contamount, "주문번호:", self.ordernum, self.etc)
 
 
 
# # CpEvent: 실시간 이벤트 수신 클래스
# class CpEvent:
#     def set_params(self, client, name, parent):
#         self.client = client   # CP 실시간 통신 object
#         self.name = name       # 서비스가 다른 이벤트를 구분하기 위한 이름
#         self.parent = parent   # callback 을 위해 보관
 
#         # 데이터 변환용
#         self.concdic = {"1" : "체결", "2" : "확인", "3" : "거부", "4" : "접수"}
#         self.buyselldic = {"1" : "매도", "2" : "매수"}
#         print(self.concdic)
#         print(self.buyselldic)
 
#     # PLUS 로 부터 실제로 시세를 수신 받는 이벤트 핸들러 
#     def OnReceived(self):
#         print(self.name)
#         if self.name == "stockcur" :
#             # 현재가 체결 데이터 실시간 업데이트
#             exFlag = self.client.GetHeaderValue(19)  # 예상체결 플래그
#             cprice = self.client.GetHeaderValue(13)  # 현재가
#             # 장중이 아니면 처리 안함.
#             if exFlag != ord('2'):
#                 return
 
#             # 현재가 업데이트
#             self.parent.sprice.cur = cprice
#             print("PB > 현재가 업데이트 : ", cprice)
 
#             # 현재가 변경  call back 함수 호출
#             self.parent.monitorPriceChange()
 
#             return
 
#         elif self.name == "stockbid" :
#             # 현재가 10차 호가 데이터 실시간 업데이트
#             dataindex = [3,4,7,8,11,12, 15,16, 19, 20, 27,28, 31,32,35,36,39,40,43,44]
#             obi = 0
#             for i in range(10):
#                 self.parent.sprice.offer[i] = self.client.GetHeaderValue(dataindex[obi])
#                 self.parent.sprice.bid[i] = self.client.GetHeaderValue(dataindex[obi + 1])
#                 obi += 2
 
#             # for debug
#             for i in range(10):
#                 print("PB > 10차 호가 : ",i + 1, "차 매도/매수 호가: ", self.parent.sprice.offer[i], self.parent.sprice.bid[i])
#             return True
 
#             # 10차 호가 변경 call back 함수 호출
#             self.parent.monitorPriceChange()
 
#             return
 
#         elif self.name == "conclution" :
#             # 주문 체결 실시간 업데이트
#             conflag = self.client.GetHeaderValue(14)    # 체결 플래그
#             ordernum = self.client.GetHeaderValue(5)    # 주문번호
#             amount = self.client.GetHeaderValue(3)      # 체결 수량
#             price = self.client.GetHeaderValue(4)       # 가격
#             code = self.client.GetHeaderValue(9)        # 종목코드
#             bs = self.client.GetHeaderValue(12)         # 매수/매도 구분
#             balace = self.client.GetHeaderValue(23)  # 체결 후 잔고 수량
 
#             conflags = ""
#             if conflag in self.concdic :
#                 conflags = self.concdic.get(conflag)
#                 print(conflags)
 
#             bss = ""
#             if (bs in self.buyselldic):
#                 bss = self.buyselldic.get(bs)
 
#             print(conflags, bss, code, "주문번호:", ordernum)
#             # call back 함수 호출해서 orderMain 에서 후속 처리 하게 한다.
#             self.parent.monitorOrderStatus(code, ordernum, conflags, price, amount, balace)
#             return
 
 
# # CpPBStockCur: 실시간 현재가 요청 클래스
# class CpPBStockCur:
#     def __init__(self):
#         self.name = "stockcur"
#         self.obj = win32com.client.Dispatch("DsCbo1.StockCur")
 
#     def Subscribe(self, code, sprice, parent):
#         self.obj.SetInputValue(0, code)
#         handler = win32com.client.WithEvents(self.obj, CpEvent)
#         handler.set_params(self.obj, self.name, parent)
#         self.obj.Subscribe()
#         self.sprice = sprice
 
#     def Unsubscribe(self):
#         self.obj.Unsubscribe()
 
# # CpPBStockBid: 실시간 10차 호가 요청 클래스
# class CpPBStockBid:
#     def __init__(self):
#         self.name = "stockbid"
#         self.obj = win32com.client.Dispatch("Dscbo1.StockJpBid")
 
#     def Subscribe(self, code, sprice, parent):
#         self.obj.SetInputValue(0, code)
#         handler = win32com.client.WithEvents(self.obj, CpEvent)
#         handler.set_params(self.obj, self.name, parent)
#         self.obj.Subscribe()
#         self.sprice = sprice
 
 
#     def Unsubscribe(self):
#         self.obj.Unsubscribe()
 
# # CpPBConclusion: 실시간 주문 체결 수신 클래그
# class CpPBConclusion:
#     def __init__(self):
#         self.name = "conclution"
#         self.obj = win32com.client.Dispatch("DsCbo1.CpConclusion")
 
#     def Subscribe(self, parent):
#         self.parent = parent
#         handler = win32com.client.WithEvents(self.obj, CpEvent)
#         handler.set_params(self.obj, self.name, parent)
#         self.obj.Subscribe()
 
#     def Unsubscribe(self):
#         self.obj.Unsubscribe()
 
# class CpRPOrder:
#     def __init__(self):
#         # 연결 여부 체크
#         self.objCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")
#         bConnect = self.objCpCybos.IsConnect
#         if (bConnect == 0):
#             print("PLUS가 정상적으로 연결되지 않음. ")
#             return
 
#         # 주문 초기화
#         self.objTrade = win32com.client.Dispatch("CpTrade.CpTdUtil")
#         initCheck = self.objTrade.TradeInit(0)
#         if (initCheck != 0):
#             print("주문 초기화 실패")
#             return
        
#         self.acc = self.objTrade.AccountNumber[0]  # 계좌번호
#         self.accFlag = self.objTrade.GoodsList(self.acc, 1)  # 주식상품 구분
#         print(self.acc, self.accFlag[0])
 
#         # 매수/정정/취소 주문 object 생성
#         self.objBuyOrder = win32com.client.Dispatch("CpTrade.CpTd0311")     # 매수
#         self.objModifyOrder = win32com.client.Dispatch("CpTrade.CpTd0313")  # 정정
#         self.objCancelOrder = win32com.client.Dispatch("CpTrade.CpTd0314")  # 취소
#         self.orderNum = 0 # 주문 번호
 
#     def buyOrder(self, code, price, amount):
#         # 주식 매수 주문
#         print("신규 매수", code, price, amount)
 
#         self.objBuyOrder.SetInputValue(0, "2")  # 2: 매수
#         self.objBuyOrder.SetInputValue(1, self.acc)  # 계좌번호
#         self.objBuyOrder.SetInputValue(2, self.accFlag[0])  # 상품구분 - 주식 상품 중 첫번째
#         self.objBuyOrder.SetInputValue(3, code)  # 종목코드
#         self.objBuyOrder.SetInputValue(4, amount)  # 매수수량
#         self.objBuyOrder.SetInputValue(5, price)  # 주문단가 
#         self.objBuyOrder.SetInputValue(7, "0")  # 주문 조건 구분 코드, 0: 기본 1: IOC 2:FOK
#         self.objBuyOrder.SetInputValue(8, "01")  # 주문호가 구분코드 - 01: 보통
 
#         # 매수 주문 요청
#         self.objBuyOrder.BlockRequest()
 
#         rqStatus = self.objBuyOrder.GetDibStatus()
#         rqRet = self.objBuyOrder.GetDibMsg1()
#         print("통신상태", rqStatus, rqRet)
#         if rqStatus != 0:
#             return False
 
#         # 주의: 매수 주문에  대한 구체적인 처리는 cpconclution 으로 파악해야 한다.
#         return True
 
#     def modifyOrder(self, ordernum, code, price):
#         # 주식 정정 주문
#         print("정정주문", ordernum, code, price)
#         self.objModifyOrder.SetInputValue(1, ordernum)  #  원주문 번호 - 정정을 하려는 주문 번호
#         self.objModifyOrder.SetInputValue(2, self.acc)  # 상품구분 - 주식 상품 중 첫번째
#         self.objModifyOrder.SetInputValue(3, self.accFlag[0])  # 상품구분 - 주식 상품 중 첫번째
#         self.objModifyOrder.SetInputValue(4, code)  # 종목코드
#         self.objModifyOrder.SetInputValue(5, 0)  # 정정 수량, 0 이면 잔량 정정임
#         self.objModifyOrder.SetInputValue(6, price)  #  정정주문단가
 
#         # 정정주문 요청
#         self.objModifyOrder.BlockRequest()
 
#         rqStatus = self.objModifyOrder.GetDibStatus()
#         rqRet = self.objModifyOrder.GetDibMsg1()
#         print("통신상태", rqStatus, rqRet)
#         if rqStatus != 0:
#             return False
 
#         # 새로운 주문 번호 구한다.
#         self.orderNum = self.objModifyOrder.GetHeaderValue(7)
 
#     def cancelOrder(self, ordernum, code):
#         # 주식 취소 주문
#         print("취소주문", ordernum, code)
#         self.objCancelOrder.SetInputValue(1, ordernum)  #  원주문 번호 - 정정을 하려는 주문 번호
#         self.objCancelOrder.SetInputValue(2, self.acc)  # 상품구분 - 주식 상품 중 첫번째
#         self.objCancelOrder.SetInputValue(3, self.accFlag[0])  # 상품구분 - 주식 상품 중 첫번째
#         self.objCancelOrder.SetInputValue(4, code)  # 종목코드
#         self.objCancelOrder.SetInputValue(5, 0)  # 정정 수량, 0 이면 잔량 취소임
 
#         # 취소주문 요청
#         self.objCancelOrder.BlockRequest()
 
#         rqStatus = self.objCancelOrder.GetDibStatus()
#         rqRet = self.objCancelOrder.GetDibMsg1()
#         print("통신상태", rqStatus, rqRet)
#         if rqStatus != 0:
#             return False
 
# # CpRPCurrentPrice : 주식 현재가 및 10차 호가 조회
# class CpRPCurrentPrice:
#     def __init__(self):
#         self.objCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")
#         bConnect = self.objCpCybos.IsConnect
#         if (bConnect == 0):
#             print("PLUS가 정상적으로 연결되지 않음. ")
#             return
#         self.objStockMst = win32com.client.Dispatch("DsCbo1.StockMst")
#         self.objStockjpbid = win32com.client.Dispatch("DsCbo1.StockJpBid2")
 
#         return
 
 
#     def Request(self, code, rtMst):
#         # 현재가 통신
#         self.objStockMst.SetInputValue(0, code)
#         self.objStockMst.BlockRequest()
 
#         # 10차 호가 통신
#         self.objStockjpbid.SetInputValue(0, code)
#         self.objStockjpbid.BlockRequest()
 
#         print("통신상태", self.objStockMst.GetDibStatus(), self.objStockMst.GetDibMsg1())
#         if self.objStockMst.GetDibStatus() != 0:
#             return False
#         print("통신상태", self.objStockjpbid.GetDibStatus(), self.objStockjpbid.GetDibMsg1())
#         if self.objStockjpbid.GetDibStatus() != 0:
#             return False
 
#         # 수신 받은 현재가 정보를 rtMst 에 저장
#         rtMst.cur =  self.objStockMst.GetHeaderValue(11)  # 종가
#         # 10차호가
#         for i in range(10):
#             rtMst.offer.append(self.objStockjpbid.GetDataValue(0, i))  # 매도호가
#             rtMst.bid.append(self.objStockjpbid.GetDataValue(1, i) ) # 매수호가
 
#         # for debug
#         for i in range(10):
#             print(i+1, "차 매도/매수 호가: ", rtMst.offer[i], rtMst.bid[i])
#         return True
 
 
# # 주문 테스트용 클래스
# class OrderMain() :
#     def __init__(self):
#         self.isSB = False   # 실시간 처리        
#         self.initOrder()    # 주문 상태 - 초기화
 
#         self.sprice = stockPricedData()  # 주문 현재가/10차 호가 저장 (실시간 업데이트)
#         self.cporder = CpRPOrder()    # 주문 통신 object
 
#         # 실시간 통신 object
#         self.cur = CpPBStockCur()
#         self.bid = CpPBStockBid()
 
#         # 주문체결은 미리 실시간 요청
#         self.conclution = CpPBConclusion()
#         self.conclution.Subscribe(self)
 
#         self.history = []
 
#     def stopSubscribe(self):
#         if self.isSB:
#             self.cur.Unsubscribe()
#             self.bid.Unsubscribe()
 
#         self.isSB = False
 
#     # 매수 주문
#     def BuyOrder(self):
#         self.stopSubscribe()
#         self.code = "A003540"  # 테스트용 종목코드
#         self.buyamount = 1     # 주문 수량
 
#         # 1. 현재가 구하기
#         price = CpRPCurrentPrice()
#         if price.Request(self.code, self.sprice) == False:
#             print("현재가 통신 실패")
#             self.initOrder()
#             return
 
 
#         print("신규 매수주문 - ", self.orderNonce + 1, "차 매수호가 ", + self.sprice.bid[self.orderNonce])
#         bResult = self.cporder.buyOrder(self.code,self.sprice.bid[self.orderNonce], self.buyamount)
#         if bResult == False:
#             print("주문 실패")
#             self.initOrder()
#             return
#         self.orderStatus = orderStatus.newOrder  # 주문상태 업데이트
 
#         # 실시간 통신 요청
#         self.cur.Subscribe(self.code, self.sprice, self)
#         self.bid.Subscribe(self.code, self.sprice, self)
#         self.isSB = True
 
#     # 정정주문
#     def ModifyOrder(self):
#         if not(self.orderStatus == orderStatus.orderConfirm):
#             print("정정주문 확인 불가 상태 ")
#             return
 
#         if self.ordernum == 0:
#             print("주문 번호가 없습니다")
#             return
 
#         # 정정주문 할 때 마다 1 호가씩 올린다.
#         self.orderNonce -= 1
#         if self.orderNonce <= 0:
#             self.orderNonce = 0
#         print("정정 주문 - ", self.orderNonce + 1, "차 매수호가 ", + self.sprice.bid[self.orderNonce])
#         bResult = self.cporder.modifyOrder(self.ordernum, self.code, self.sprice.bid[self.orderNonce])
#         if  bResult== False:
#             print("정정 주문 실패")
#             return
 
#         # 주문상태 업데이트
#         self.orderStatus = orderStatus.modifyOrder
 
#         # 정정주문은 거래소에서 거부 당할 수 있어 확인/거부 여부를 반드시 확인 해야 함.
 
#         return
 
 
#     # 취소주문
#     def CancelOrder(self):
#         if not(self.orderStatus == orderStatus.orderConfirm):
#             print("취소주문 확인 불가 상태 ")
#             return
 
#         if self.ordernum == 0:
#             print("주문 번호가 없습니다")
#             return
#         # 취소 주문
#         bResult = self.cporder.cancelOrder(self.ordernum, self.code)
#         if bResult == False:
#             print("취소 주문 실패")
#             return
 
#         self.orderStatus = orderStatus.cancelOrder  # 주문상태 업데이트
#         # 취소주문은 거래소에서 거부 당할 수 있어 확인/거부 여부를 반드시 확인 해야 함.
 
#         return
 
#     # 전체 클리어
#     def clearAll(self):
#         self.initOrder()
#         self.stopSubscribe()
#         self.conclution.Unsubscribe()
 
#         # debug
#         if (len(self.history)) :
#             print("주문 내역 정리 ============================")
#             for i in range(0, len(self.history)) :
#                 self.history[i].printhistory()
 
#         self.history = []
 
#         return
 
#     def initOrder(self):
#         # 주문 정보 초기화
#         self.orderStatus = orderStatus.nothing
#         self.ordernum = 0        # 주문번호
#         self.remainAmount = 0 # 주문 후 미체결 수량
#         self.orderNonce = 9     # 매수 주문 호가 조정 변수 ( 9 > 8 > 7 .. 순으로 호가 조정)
 
#     def monitorPriceChange(self):
#         # 이곳에서 시세 변경에 대한 감시 등의 로직 추가고려
 
#         return
 
 
#     # 실시간 주문 체결 업데이트
#     def monitorOrderStatus(self, code, ordernum, conflags, price, amount, balance) :
#         print("주문체결: ", code, ordernum, conflags, price, amount, balance)
#         if self.orderStatus == orderStatus.nothing :
#             return
#         # 체결: 체결 시 체결 수량/미체결 수량 계산
#         if conflags == "체결":
#             self.remainAmount -= amount   # 미체결 수량 계산
#             if self.orderStatus == orderStatus.orderConfirm :
#                 print("주문 체결 됨 ", "수량", amount, "잔고수량:", balance, "미체결수량:", self.remainAmount)
 
#             if self.remainAmount <= 0 :   # 전량 체결 됨
#                 self.initOrder()
 
#             # for debug
#             history = orderHistoryData()
#             history.sethistory(conflags, code, price, self.remainAmount, amount, ordernum, "")
#             self.history.append(history)
 
 
#         #  접수: 신규 주문 > 접수 ;--> 주문번호, 주문 정상 처리
#         elif conflags == "접수":
#             if self.orderStatus == orderStatus.newOrder:
#                 self.ordernum = ordernum  # 주문번호 업데이트
#                 self.remainAmount = amount  # 주문 후 미체결 수량
#                 self.orderStatus = orderStatus.orderConfirm
 
#                 # for debug
#                 history = orderHistoryData()
#                 history.sethistory(conflags, code, price, amount, 0, ordernum, "신규 매수")
#                 self.history.append(history)
#                 history.printhistory()
 
#         #  확인: 정정/취소 주문 > 확인 ;--> 정정/취소 주문 정상 처리 확인
#         elif conflags == "확인":
#             etc = ""
#             if self.orderStatus == orderStatus.modifyOrder:     # 정정 확인
#                 self.ordernum = ordernum  # 주문번호 업데이트
#                 self.orderStatus = orderStatus.orderConfirm
#                 etc  = "정정확인"
#             elif self.orderStatus == orderStatus.cancelOrder:    # 취소 확인
#                 self.initOrder()
#                 etc  = "취소확인"
 
#             # for debug
#             history = orderHistoryData()
#             print(code, price)
#             print(self.remainAmount, ordernum)
#             history.sethistory(conflags, code, price, self.remainAmount, 0,  ordernum, etc)
#             self.history.append(history)
#             history.printhistory()
 
 
#         # 거부: 정정/취소 주문 > 거부 ;--> 정정/취소 주문 거부, 정정/취소 불가
#         elif conflags == "거부":
#             if self.orderStatus == orderStatus.modifyOrder or self.orderStatus == orderStatus.cancelOrder:
#                 print("주문거부 발생, 반드시 확인 필요")
#                 self.orderStatus = orderStatus.newOrder  # 주문 상태를 이전으로 돌림
 
#             # for debug
#             history = orderHistoryData()
#             history.sethistory(conflags, code, price, amount, 0,  ordernum, "")
#             self.history.append(history)
#             history.printhistory()
 
 
#         return
 
 
# class MyWindow(QMainWindow):
 
#     def __init__(self):
#         self.orerMain = OrderMain()
#         super().__init__()
#         self.setWindowTitle("PLUS API TEST")
#         self.setGeometry(300, 300, 300, 230)
 
#         btnBuy = QPushButton("매수주문", self)
#         btnBuy.move(20, 20)
#         btnBuy.resize(200,30)
#         btnBuy.clicked.connect(self.btnBuy_clicked)
 
#         btnModify = QPushButton("정정주문(1호가씩 올림)", self)
#         btnModify.move(20, 70)
#         btnModify.resize(200,30)
#         btnModify.clicked.connect(self.btnModify_clicked)
 
#         btnCancel = QPushButton("취소주문", self)
#         btnCancel.move(20, 120)
#         btnCancel.resize(200,30)
#         btnCancel.clicked.connect(self.btnCancel_clicked)
 
#         btnExit = QPushButton("종료", self)
#         btnExit.move(20, 170)
#         btnExit.resize(200,30)
#         btnExit.clicked.connect(self.btnExit_clicked)
 
 
#     # 매수 주문
#     def btnBuy_clicked(self):
#         self.orerMain.BuyOrder()
 
#     # 정정주문
#     def btnModify_clicked(self):
#         self.orerMain.ModifyOrder()
#         return
 
 
#     # 취소주문
#     def btnCancel_clicked(self):
#         self.orerMain.CancelOrder()
 
#     # 종료
#     def btnExit_clicked(self):
#         self.orerMain.clearAll()
#         exit()
 
 
# if __name__ == "__main__":
#     app = QApplication(sys.argv)
#     myWindow = MyWindow()
#     myWindow.show()
#     app.exec_()


---

In [21]:
import win32com.client
import time
from datetime import datetime
import redis
import random
import os

 # 인스턴스
instCpCybos = win32com.client.Dispatch('CpUtil.CpCybos')  #사이보스플러스 연결 상태를 확인하는 인스턴스
instStockChart = win32com.client.Dispatch("CpSysDib.StockChart")  #차트데이터를 수신하는 인스턴스
instCpStockCode = win32com.client.Dispatch("CpUtil.CpStockCode")  #주식 종목명 및 코드를 조회하는 인스턴스
instCpCodeMgr = win32com.client.Dispatch(("CpUtil.CpCodeMgr"))  #각종 코드정보 및 코드리스트를 얻는 인스턴스
instMarketEye = win32com.client.Dispatch("CpSysDib.MarketEye")  # 여러종목의 필요항목을 한번에 수신하는 인스턴스
instCpTdUtil = win32com.client.Dispatch("CpTrade.CpTdUtil")  # 계좌정보
instCpTd0311 = win32com.client.Dispatch("CpTrade.CpTd0311")  # 매수 매도 주문 인스턴스
instCpTdNew5331A = win32com.client.Dispatch("CpTrade.CpTdNew5331A")  # 계좌별 매수 가능 금액/ 수량 데이터
instCpTdNew5331B = win32com.client.Dispatch("CpTrade.CpTdNew5331B")  # 계좌별 매도 가능 수량데이터
instCpTd6033 = win32com.client.Dispatch("CpTrade.CpTd6033")  # 계좌별 잔고 및 주문체결 평가현황데이터를 요청하고 수신한다
instStockMst = win32com.client.Dispatch("Dscbo1.StockMst")  # 현재가, 호가 관련
instStockWeek = win32com.client.Dispatch("DsCbo1.StockWeek")  # 일별 데이터
instCpConclusion = win32com.client.Dispatch("Dscbo1.CpCOnclusion")  # 주식 체결 실시간


# 계좌정보
instCpTdUtil.TradeInit()
accountNumber = instCpTdUtil.AccountNumber[0]  # 계좌번호
accFlag = instCpTdUtil.GoodsList(accountNumber,1)  #거래구분 코드
 
# 시간 정보
now = int(datetime.now().strftime('%H%M'))
 
#print(hour)
 
# 매도 셋팅 / 손절선 0.95 / 익절선 1.05
 
# 연결 확인
def connect_check():
    print('사이보스플러스 연결를 체크합니다. 주인님')
    bConnect = instCpCybos.IsConnect
    if (bConnect == 0):
        print("PLUS가 정상적으로 연결되지 않았습니다요. ")
        exit() # 프로그램 종료
    elif (bConnect == 1):
        print('접속 성공입니다요. 주인님')


 # target_list 필터링 / save / read 함수

def target_list_save_txt():  #target_list.txt 저장 / target_list_save.txt 함수는 장 전에 미리 수동실행 또는 자동실행 조건 설정
    f = open('C:\\Users\\green\\OneDrive\\바탕 화면\\target_list.txt', 'wt')
    target_list = cap_filter(instStockChart)
    for code in target_list:
        f.writelines('%s\n' %code)
    f.close()
    print('target_list.txt 저장 완료입니다요')
 
def target_list_read_txt(): # 저장된 target_list.txt 를 읽어오는 함수
    target_list = []
    f = open('C:\\Users\\green\\OneDrive\\바탕 화면\\target_list.txt', 'rt')
    target_list_a = f.readlines()
    for i in target_list_a:
        a = i.split('\n')[0]
        target_list.append(a)
    return target_list

In [22]:
shcode_list = ['064350', '017670', '005490', '014680', '012450', '051900', '009150', '066570', '108320', '047810', '000660', '010140', '035420', '005380', '009830', '052690', '034020', '005070', '005420', '042700', '028050', '068270', '137310', '005930', '017960', '075580', '003490']
 
def market_shcode():  #target_list_save_txt함수 호출 시 연동됨 / 코스피,코스닥 중 주권 종목의 코드를 리스트로 추출함
    # kospilist = instCpCodeMgr.GetStockListByMarket(1)   #코스피 코드리스트
    # kosdaqlist = instCpCodeMgr.GetStockListByMarket(2)  #코스닥 코드리스트

    # market = kospilist + kosdaqlist  # 코스피 + 코스닥 코드리스트
    # market_stock = [] # => market_shcode = [] 

    market_shcode = []
    for code in shcode_list:
        check = instCpCodeMgr.GetStockSectionKind(code)
        if check == 1:
            market_shcode.append(code)
    return market_shcode



In [23]:
def cap_filter(instStockChart):   #market_shcode을 시가총액으로 필터링 / market_shcode()을 호출하여 시가총액으로 필터링
    market_code = market_shcode()  #target_list_save_txt함수 호출 시 연동됨 /
    target_list = []
    print('시가총액 300억~1000억 종목을 찾고 있습니다. 주인님')
    for code in market_code:
        instStockChart.SetinputValue(0,code)
        instStockChart.SetinputValue(1,ord('2'))
        instStockChart.SetinputValue(4,1)
        instStockChart.SetinputValue(5,13)
        instStockChart.SetinputValue(6,ord('m'))
        instStockChart.SetinputValue(9,ord('1'))
        instStockChart.BlockRequest()
        time.sleep(1)
 
        cap =  instStockChart.GetDataValue(0,0)
 
        if (cap >= 30000000000) and (cap <= 100000000000):
            target_list.append(code)
            print(len(target_list),'/',len(market_code))
 
    print('타겟 리스트는',len(target_list),'개 입니다. 주인님')
    return target_list



In [24]:
 
# 보유종목 반환 함수
def bought_item():  # 보유종목 정보를 반환하는 함수  -> 종목보유여부 체크 및 매도 감시 시 활용
    instCpTdNew5331B.SetInputValue(0,accountNumber)
    instCpTdNew5331B.SetInputValue(1,accFlag[0])
    instCpTdNew5331B.SetInputValue(3,ord('1'))
    instCpTdNew5331B.SetInputValue(4,ord('1'))
    instCpTdNew5331B.BlockRequest()
 
    try:
        savestock_code = instCpTdNew5331B.GetDataValue(0,0)  # 보유종목 코드
        savestock_name = instCpTdNew5331B.GetDataValue(1,0)  # 보유종목 이름
        savestock_quant = instCpTdNew5331B.GetDataValue(12,0) #보유종목 매도 가능 수량
        return (savestock_name,savestock_quant,savestock_code)
    except:
        savestock_code = 'none'
        savestock_name = 'none'
        savestock_quant = 0
        return (savestock_name,savestock_quant,savestock_code)
 
def bought_check(): # 보유종목이 있는지 체크하는 함수
    instCpTdNew5331B.SetInputValue(0, accountNumber)
    instCpTdNew5331B.SetInputValue(1, accFlag[0])
    instCpTdNew5331B.SetInputValue(3, ord('1'))
    instCpTdNew5331B.SetInputValue(4, ord('1'))
    instCpTdNew5331B.BlockRequest()
 
    try:
        balance = instCpTdNew5331B.GetDataValue(12, 0)  # 매도가능수량
        if balance != 0:
            return 1  #매도가능 수량이 있음
    except:
        return 0  #매도가능 수량이 없음 / 없으면 에러가 남
 


In [25]:
 
# 종목 선정 함수
def buy_observer(instStockChart,target_list):  # target_list를 서치하여 1개의 종목을 찾아냄
    check = 0
    will_buy_item = []
    while check == 0:
        time.sleep(1.02)
        for code in target_list:   # tqrget_list의 종목 별 0~11봉 거래량 값을 입력값으로 설정
            instStockChart.SetInputValue(0, code)   #종목 코드
            instStockChart.SetInputValue(1, ord('2'))   #'1' 날짜구간 , '2' 갯수
            instStockChart.SetInputValue(4, 11)   # 11개 값 수신 요청  0봉(현재봉) ~ 10봉 까지 11개 값
            instStockChart.SetInputValue(5, 8)    # 거래량
            instStockChart.SetInputValue(6, ord('m'))   # 일봉
            instStockChart.SetInputValue(9, ord('1'))   # 수정주가
            instStockChart.BlockRequest()   #입력값을 api 요청
 
            volumes = []
            numData = instStockChart.GetHeaderValue(3)   # 수신 값의 갯수, 11봉을 요청했으므로 11이 수신됨
            for i in range(numData): # 0~10까지 반복하며 0봉부터 10봉까지의 거래량을 volumes 에 리스트로 저장함
                volume = instStockChart.GetDataValue(0, i)
                volumes.append(volume)
 
            averageVolume = (sum(volumes) - volumes[0]) / (len(volumes) - 1)  # 1~10봉 거래량 평균
 
            if (volumes[0] > averageVolume * 10): #0봉(현재봉)과 1~10봉 평균 거래량으로 조건을 설정하여 will_buy_item에 바인딩
                will_buy_item.append(code)
                print('종목을 찾았습니다요. 주인님. 종목코드는',code,'입니다요')
                check = 1   # while 문 탈출
                break  # for문 탈출 순차 탐색 중 먼저 조건에 부합하는 종목을 발견하면 탐색을 종료하는 방식이므로
                       # for문의 속도가 느릴 경우 target_list의 순서 영향을 많이 받을 것임. 종목 탐색의 시간 설정이 중요함
    return will_buy_item
 


In [26]:
 
# will_buy_item 호가 출력 및 반환 함수
def item_hoga_data(will_buy_item):   # 호가 정보를 출력하는 함수 / 매수 주문 시 단가로 활용/ 1차 호가주문으로 활용해보자
    instStockMst.SetInputValue(0,will_buy_item)
    instStockMst.BlockRequest()
 
    item_buy_hoga_1 = instStockMst.GetDataValue(1,0)  # 매수 1차 호가 // 매수 주문 시 활용
    item_buy_hoga_2 = instStockMst.GetDataValue(1, 1)
    item_buy_hoga_3 = instStockMst.GetDataValue(1, 2)
    item_sell_hoga_1 = instStockMst.GetDataValue(0, 0)  # 매도 1차 호가
    item_sell_hoga_2 = instStockMst.GetDataValue(0, 1)
    item_sell_hoga_3 = instStockMst.GetDataValue(0, 2)
 
    print('호가정보입니다요.')
    print(instStockMst.GetDataValue(0,2))
    print(instStockMst.GetDataValue(0,1))
    print(instStockMst.GetDataValue(0,0))
    print('--------------------')
    print(instStockMst.GetDataValue(1,0))
    print(instStockMst.GetDataValue(1,1))
    print(instStockMst.GetDataValue(1,2))
 
    return (item_buy_hoga_1,item_buy_hoga_2,item_buy_hoga_3,
            item_sell_hoga_1,item_sell_hoga_2,item_sell_hoga_3)
 


In [27]:
#매수 정보 반환 함수
def buy_possible(will_buy_item,buy_hoga):  # 매수를 위한 정보 / 호가 정보를 인수롤 받아오도록 수정해야함.
    instCpTdNew5331A.SetInputValue(0, accountNumber) # 계좌번호
    instCpTdNew5331A.SetInputValue(1, accFlag[0]) # 상품관리 구분 코드
    instCpTdNew5331A.SetInputValue(2, will_buy_item) # 2 - (string) 종목코드[default:""]- 수량조회시입력
    instCpTdNew5331A.SetInputValue(3, '01') #일반/ fok로 주문하면될듯
    instCpTdNew5331A.SetInputValue(4, buy_hoga) # 4 - (long) 주문단가[default:0] - 수량조회시입력
    instCpTdNew5331A.SetInputValue(5, 'Y') # 5 - Y:증거금 100 %
    instCpTdNew5331A.SetInputValue(6, ord('2')) # 6 - (char)조회구분코드; '1'=금액조회[default], '2'=수량조회
    instCpTdNew5331A.BlockRequest()
 
    possible_amount = int(instCpTdNew5331A.GetHeaderValue(9)) # 증거금 100% 가능 금액 조회
    possible_quant = int(instCpTdNew5331A.GetHeaderValue(17)) # 증거금 100% 주문 가능 수량 조회
 
    print('주문가능금액과',possible_amount)
    print('주문가능수량',possible_quant,'입니다요')
    return (possible_quant,possible_amount)
 
#매수 주문 함수
def buy_order(item,quant,hoga): # 매수주문
    instCpTd0311.SetInputValue(0,'2')
    instCpTd0311.SetInputValue(1,accountNumber)
    instCpTd0311.SetInputValue(2,accFlag[0])
    instCpTd0311.SetInputValue(3,item)
    instCpTd0311.SetInputValue(4,quant)
    instCpTd0311.SetInputValue(5,hoga)
    instCpTd0311.SetInputValue(7,'2') #FOK
    instCpTd0311.SetInputValue(8,'01') #01보통 03시장가 13최우선지정가
    instCpTd0311.BlockRequest()
 
#계좌 수익률 반환 함수
def benefit_ratio():  # 계좌 수익률을 출력하는 함수 / 매도 감시 시 활용
 
    instCpTd6033.SetInputValue(0,accountNumber)
    instCpTd6033.SetInputValue(1,accFlag[0])
    instCpTd6033.SetInputValue(3,'1')
    instCpTd6033.BlockRequest()
 
    #print(instCpTd6033.GetHeaderValue(8))  # 수익률 / 수익률을 기준으로 손절 익절 매도주문을 진행 / 수수료 감안해야함
    ratio = instCpTd6033.GetHeaderValue(8)
    return ratio
 
#호가 출력 및 반환 함수
def save_hoga_data(savestock_code):  # 호가 정보를 출력하는 함수 / 매도 주문 시 단가로 활용/ 1차 호가주문으로 활용해보자
 
    instStockMst.SetInputValue(0, savestock_code)
    instStockMst.BlockRequest()
 
    save_buy_hoga_1 = instStockMst.GetDataValue(1, 0)  # 매수 1차 호가 // 매수 주문 시 활용
    save_buy_hoga_2 = instStockMst.GetDataValue(1, 1)
    save_buy_hoga_3 = instStockMst.GetDataValue(1, 2)
    save_sell_hoga_1 = instStockMst.GetDataValue(0, 0)  # 매도 1차 호가
    save_sell_hoga_2 = instStockMst.GetDataValue(0, 1)
    save_sell_hoga_3 = instStockMst.GetDataValue(0, 2)
 
    print('호가정보입니다요.')
    print(instStockMst.GetDataValue(0,2))
    print(instStockMst.GetDataValue(0,1))
    print(instStockMst.GetDataValue(0,0))
    print('--------------------')
    print(instStockMst.GetDataValue(1,0))
    print(instStockMst.GetDataValue(1,1))
    print(instStockMst.GetDataValue(1,2))
 
    return (save_buy_hoga_1,save_buy_hoga_2,save_buy_hoga_3,
            save_sell_hoga_1,save_sell_hoga_2,save_sell_hoga_3)
 
# 매도감시 함수 / 수익률이 익절선, 손절선 만족 시 매도 주문 실행 / 장중에만 실행시키려면?
def sell_obsever():
    print('매도 감시중입니다요. 주인님')
    check = 0
    #ratio_count = 0  # 임시로 무한루프를 종료하기 위한 기능
    while check == 0:
        time.sleep(1)
        ratio = benefit_ratio()
        print('현재수익률입니다요:',ratio)
        #ratio_count += 1
        #if ratio_count == 10:
         #   print('종료처리합니다요')
          #  check = 1
           # return 1
        if ratio > 10: # 익절
            print('얏호 익절 신호입니다요')
            check = 1
            return 1
        elif ratio < 7: # 손절
            print('손절입니다 도망치십시요~!')
            check = 1
            return 0
 
 
#매도주문 함수
def sell_order(item,quant,hoga):  # 매도주문
    instCpTd0311.SetInputValue(0,'1') # 매도
    instCpTd0311.SetInputValue(1,accountNumber)
    instCpTd0311.SetInputValue(2,accFlag[0])
    instCpTd0311.SetInputValue(3,item)
    instCpTd0311.SetInputValue(4,quant)
    instCpTd0311.SetInputValue(5,hoga)
    instCpTd0311.SetInputValue(7,'2')
    instCpTd0311.SetInputValue(8,'01')
    instCpTd0311.BlockRequest()
 
 
#프로그램 실행
 
#연결 여부 체크
connect_check()
 
#타겟 리스트 셋팅
now = int(datetime.now().strftime('%H%M'))
if now < 810:
    target_list_save_txt()
 
target_list = target_list_read_txt()  # target_list 호출 //
 
now = int(datetime.now().strftime('%H%M'))
 

while now < 901 or now > 1530:
    now = int(datetime.now().strftime('%H%M'))
    print('장 중이 아닙니다요.')
    time.sleep(15)
 
 
while 1:
    boughtcheck = bought_check()  # 0:미보유중 1:보유중
    #보유 종목이 있을 경우 매도감시 및 매도주문 / 매도주문 완료 시 프로그램 종료
    if boughtcheck == 1:
        print('보유종목 정보를 가져와보겠습니다요')
        savestock_name,savestock_quant,savestock_code = bought_item()
        print(savestock_name,'종목을',savestock_quant,'주 보유중입니다요')
        #장중일 경우 매도 감시/ 장중인 것은 어떻게 처리할 것인가
        sell_check = sell_obsever()
        if sell_check == 1 : #익절 신호 확인 시 매도 정보 추출
            print('익절 신호입니다요!!') # 매수 1호가에 익절
            save_buy_hoga_1, save_buy_hoga_2, save_buy_hoga_3,\
            save_sell_hoga_1, save_sell_hoga_2, save_sell_hoga_3\
            = save_hoga_data(savestock_code)
            print(savestock_name,'종목',savestock_quant,'주를',save_buy_hoga_1,
                  '원에 매도 주문 신청합니다요')
            instCpTdUtil.TradeInit()
            sell_order(savestock_code,savestock_quant,save_buy_hoga_1)
            print('매도 주문 완료입니다 주인님')
            print('먹은 돈 가지고 튑니다. 프로그램을 종료합니다')
            quit()

# ==================================================================
 
        elif sell_check == 0: # 손절 신호 확인 시 매도 정보 추출
            print('손실입니다요!!')
            save_buy_hoga_1, save_buy_hoga_2, save_buy_hoga_3, \
            save_sell_hoga_1, save_sell_hoga_2, save_sell_hoga_3 \
            = save_hoga_data(savestock_code)
            print(savestock_name, '종목', savestock_quant, '주를', save_buy_hoga_1,
                  '원에 매도 주문 신청합니다요')
            instCpTdUtil.TradeInit()
            sell_order(savestock_code,savestock_quant,save_buy_hoga_1)
            print('매도 주문 완료입니다요 주인님')
            print('오늘은 잃었습니다요 ㅜ')
            print('내일을 기약하며 프로그램 종료합니다요')
            quit()
 
    if boughtcheck == 0 : # 보유종목이 없을 경우 장 전이라면 매수 종목 탐색
        will_buy_item = buy_observer(instStockChart,target_list)
        will_buy_item_name = instCpStockCode.CodeToName(will_buy_item[0])
        item_buy_hoga_1, item_buy_hoga_2, item_buy_hoga_3,\
        item_sell_hoga_1, item_sell_hoga_2, item_sell_hoga_3\
        =item_hoga_data(will_buy_item[0])
        possible_quant,possible_amount = buy_possible(will_buy_item[0],item_sell_hoga_1)
 
        if possible_quant == 0:  #will_buy_item을 리스트로 활용할지 한 개의 종목만 담을지 //
            print('총알이 부족합니다요,')
            print('다른 종목을 찾습니다요')
            target_list.remove(will_buy_item[0])
            if len(target_list) == 0:
                print('살 수 있는 종모기 없읍니다요 ㅜㅜ')
                quit()
 
            continue
 
        if possible_quant > 0:
            print('매수를 진행합니다요')
            print(will_buy_item_name,'종목',possible_quant,'주를',
                  item_sell_hoga_1,'원에 매수합니다요')
            buy_order(will_buy_item[0],possible_quant,item_sell_hoga_1)
            print('매수주문을 완료하였습니다요')
 
            print('체결 완료 대기중입니다요')
            buy_complete_check = 0
            while buy_complete_check == 0 :
                savestock_name, savestock_quant, savestock_code\
                =bought_item()
                time.sleep(1)
                if possible_quant == savestock_quant:
                    buy_complete_check = 1
                    print('매수 체결 완료입니다요.')
 
                    continue



사이보스플러스 연결를 체크합니다. 주인님
접속 성공입니다요. 주인님
보유종목 정보를 가져와보겠습니다요
한화에어로스페이스 종목을 1 주 보유중입니다요
매도 감시중입니다요. 주인님


현재수익률입니다요: 99.61773700305811
얏호 익절 신호입니다요
익절 신호입니다요!!
호가정보입니다요.
130500
130400
130300
--------------------
130200
130100
130000
한화에어로스페이스 종목 1 주를 130200 원에 매도 주문 신청합니다요
매도 주문 완료입니다 주인님
먹은 돈 가지고 튑니다. 프로그램을 종료합니다
보유종목 정보를 가져와보겠습니다요
한화에어로스페이스 종목을 1 주 보유중입니다요
매도 감시중입니다요. 주인님
현재수익률입니다요: 99.54128440366972
얏호 익절 신호입니다요
익절 신호입니다요!!
호가정보입니다요.
130500
130400
130300
--------------------
130200
130100
130000
한화에어로스페이스 종목 1 주를 130200 원에 매도 주문 신청합니다요
매도 주문 완료입니다 주인님
먹은 돈 가지고 튑니다. 프로그램을 종료합니다
보유종목 정보를 가져와보겠습니다요
한화에어로스페이스 종목을 1 주 보유중입니다요
매도 감시중입니다요. 주인님
현재수익률입니다요: 99.61773700305811
얏호 익절 신호입니다요
익절 신호입니다요!!
호가정보입니다요.
130500
130400
130300
--------------------
130200
130100
130000
한화에어로스페이스 종목 1 주를 130200 원에 매도 주문 신청합니다요
매도 주문 완료입니다 주인님
먹은 돈 가지고 튑니다. 프로그램을 종료합니다
보유종목 정보를 가져와보겠습니다요
한화에어로스페이스 종목을 1 주 보유중입니다요
매도 감시중입니다요. 주인님
현재수익률입니다요: 99.54128440366972
얏호 익절 신호입니다요
익절 신호입니다요!!
호가정보입니다요.
130500
130400
130300
--------------------
130200
130100
130000
한화에어로스페이스 종목 1 주를 130200 원에 매도 주문 신청합니다요
매도 주문 완료입니다 주인님
먹은 

KeyboardInterrupt: 

: 

In [ ]:
# shcode_list = ['064350', '017670', '005490', '014680', '012450', '051900', '009150', '066570', '108320', '047810', '000660', '010140', '035420', '005380', '009830', '052690', '034020', '005070', '005420', '042700', '028050', '068270', '137310', '005930', '017960', '075580', '003490']
 
# def market_shcode():  #target_list_save_txt함수 호출 시 연동됨 / 코스피,코스닥 중 주권 종목의 코드를 리스트로 추출함

#     market_shcode = []
#     for code in shcode_list:
#         check = instCpCodeMgr.GetStockSectionKind(code)
#         if check == 1:
#             market_shcode.append(code)
#     return market_shcode

 
# def cap_filter(instStockChart):   #market_shcode을 시가총액으로 필터링 / market_shcode()을 호출하여 시가총액으로 필터링
#     market_code = market_shcode()  #target_list_save_txt함수 호출 시 연동됨 /
#     target_list = []
#     print('시가총액 300억~1000억 종목을 찾고 있습니다. 주인님')
#     for code in market_code:
#         instStockChart.SetinputValue(0,code)
#         instStockChart.SetinputValue(1,ord('2'))
#         instStockChart.SetinputValue(4,1)
#         instStockChart.SetinputValue(5,13)
#         instStockChart.SetinputValue(6,ord('m'))
#         instStockChart.SetinputValue(9,ord('1'))
#         instStockChart.BlockRequest()
#         time.sleep(1)
 
#         cap =  instStockChart.GetDataValue(0,0)
 
#         if (cap >= 30000000000) and (cap <= 100000000000):
#             target_list.append(code)
#             print(len(target_list),'/',len(market_code))
 
#     print('타겟 리스트는',len(target_list),'개 입니다. 주인님')
#     return target_list
 
 
# # 보유종목 반환 함수
# def bought_item():  # 보유종목 정보를 반환하는 함수  -> 종목보유여부 체크 및 매도 감시 시 활용
#     instCpTdNew5331B.SetInputValue(0,accountNumber)
#     instCpTdNew5331B.SetInputValue(1,accFlag[0])
#     instCpTdNew5331B.SetInputValue(3,ord('1'))
#     instCpTdNew5331B.SetInputValue(4,ord('1'))
#     instCpTdNew5331B.BlockRequest()
 
#     try:
#         savestock_code = instCpTdNew5331B.GetDataValue(0,0)  # 보유종목 코드
#         savestock_name = instCpTdNew5331B.GetDataValue(1,0)  # 보유종목 이름
#         savestock_quant = instCpTdNew5331B.GetDataValue(12,0) #보유종목 매도 가능 수량
#         return (savestock_name,savestock_quant,savestock_code)
#     except:
#         savestock_code = 'none'
#         savestock_name = 'none'
#         savestock_quant = 0
#         return (savestock_name,savestock_quant,savestock_code)
 
# def bought_check(): # 보유종목이 있는지 체크하는 함수
#     instCpTdNew5331B.SetInputValue(0, accountNumber)
#     instCpTdNew5331B.SetInputValue(1, accFlag[0])
#     instCpTdNew5331B.SetInputValue(3, ord('1'))
#     instCpTdNew5331B.SetInputValue(4, ord('1'))
#     instCpTdNew5331B.BlockRequest()
 
#     try:
#         balance = instCpTdNew5331B.GetDataValue(12, 0)  # 매도가능수량
#         if balance != 0:
#             return 1  #매도가능 수량이 있음
#     except:
#         return 0  #매도가능 수량이 없음 / 없으면 에러가 남
 
 
# # 종목 선정 함수
# def buy_observer(instStockChart,target_list):  # target_list를 서치하여 1개의 종목을 찾아냄
#     check = 0
#     will_buy_item = []
#     while check == 0:
#         time.sleep(1.02)
#         for code in target_list:   # tqrget_list의 종목 별 0~11봉 거래량 값을 입력값으로 설정
#             instStockChart.SetInputValue(0, code)   #종목 코드
#             instStockChart.SetInputValue(1, ord('2'))   #'1' 날짜구간 , '2' 갯수
#             instStockChart.SetInputValue(4, 11)   # 11개 값 수신 요청  0봉(현재봉) ~ 10봉 까지 11개 값
#             instStockChart.SetInputValue(5, 8)    # 거래량
#             instStockChart.SetInputValue(6, ord('m'))   # 일봉
#             instStockChart.SetInputValue(9, ord('1'))   # 수정주가
#             instStockChart.BlockRequest()   #입력값을 api 요청
 
#             volumes = []
#             numData = instStockChart.GetHeaderValue(3)   # 수신 값의 갯수, 11봉을 요청했으므로 11이 수신됨
#             for i in range(numData): # 0~10까지 반복하며 0봉부터 10봉까지의 거래량을 volumes 에 리스트로 저장함
#                 volume = instStockChart.GetDataValue(0, i)
#                 volumes.append(volume)
 
#             averageVolume = (sum(volumes) - volumes[0]) / (len(volumes) - 1)  # 1~10봉 거래량 평균
 
#             if (volumes[0] > averageVolume * 10): #0봉(현재봉)과 1~10봉 평균 거래량으로 조건을 설정하여 will_buy_item에 바인딩
#                 will_buy_item.append(code)
#                 print('종목을 찾았습니다요. 주인님. 종목코드는',code,'입니다요')
#                 check = 1   # while 문 탈출
#                 break  # for문 탈출 순차 탐색 중 먼저 조건에 부합하는 종목을 발견하면 탐색을 종료하는 방식이므로
#                        # for문의 속도가 느릴 경우 target_list의 순서 영향을 많이 받을 것임. 종목 탐색의 시간 설정이 중요함
#     return will_buy_item
 
 
# # will_buy_item 호가 출력 및 반환 함수
# def item_hoga_data(will_buy_item):   # 호가 정보를 출력하는 함수 / 매수 주문 시 단가로 활용/ 1차 호가주문으로 활용해보자
#     instStockMst.SetInputValue(0,will_buy_item)
#     instStockMst.BlockRequest()
 
#     item_buy_hoga_1 = instStockMst.GetDataValue(1,0)  # 매수 1차 호가 // 매수 주문 시 활용
#     item_buy_hoga_2 = instStockMst.GetDataValue(1, 1)
#     item_buy_hoga_3 = instStockMst.GetDataValue(1, 2)
#     item_sell_hoga_1 = instStockMst.GetDataValue(0, 0)  # 매도 1차 호가
#     item_sell_hoga_2 = instStockMst.GetDataValue(0, 1)
#     item_sell_hoga_3 = instStockMst.GetDataValue(0, 2)
 
#     print('호가정보입니다요.')
#     print(instStockMst.GetDataValue(0,2))
#     print(instStockMst.GetDataValue(0,1))
#     print(instStockMst.GetDataValue(0,0))
#     print('--------------------')
#     print(instStockMst.GetDataValue(1,0))
#     print(instStockMst.GetDataValue(1,1))
#     print(instStockMst.GetDataValue(1,2))
 
#     return (item_buy_hoga_1,item_buy_hoga_2,item_buy_hoga_3,
#             item_sell_hoga_1,item_sell_hoga_2,item_sell_hoga_3)
 
# #매수 정보 반환 함수
# def buy_possible(will_buy_item,buy_hoga):  # 매수를 위한 정보 / 호가 정보를 인수롤 받아오도록 수정해야함.
#     instCpTdNew5331A.SetInputValue(0, accountNumber) # 계좌번호
#     instCpTdNew5331A.SetInputValue(1, accFlag[0]) # 상품관리 구분 코드
#     instCpTdNew5331A.SetInputValue(2, will_buy_item) # 2 - (string) 종목코드[default:""]- 수량조회시입력
#     instCpTdNew5331A.SetInputValue(3, '01') #일반/ fok로 주문하면될듯
#     instCpTdNew5331A.SetInputValue(4, buy_hoga) # 4 - (long) 주문단가[default:0] - 수량조회시입력
#     instCpTdNew5331A.SetInputValue(5, 'Y') # 5 - Y:증거금 100 %
#     instCpTdNew5331A.SetInputValue(6, ord('2')) # 6 - (char)조회구분코드; '1'=금액조회[default], '2'=수량조회
#     instCpTdNew5331A.BlockRequest()
 
#     possible_amount = int(instCpTdNew5331A.GetHeaderValue(9)) # 증거금 100% 가능 금액 조회
#     possible_quant = int(instCpTdNew5331A.GetHeaderValue(17)) # 증거금 100% 주문 가능 수량 조회
 
#     print('주문가능금액과',possible_amount)
#     print('주문가능수량',possible_quant,'입니다요')
#     return (possible_quant,possible_amount)
 
# #매수 주문 함수
# def buy_order(item,quant,hoga): # 매수주문
#     instCpTd0311.SetInputValue(0,'2')
#     instCpTd0311.SetInputValue(1,accountNumber)
#     instCpTd0311.SetInputValue(2,accFlag[0])
#     instCpTd0311.SetInputValue(3,item)
#     instCpTd0311.SetInputValue(4,quant)
#     instCpTd0311.SetInputValue(5,hoga)
#     instCpTd0311.SetInputValue(7,'2') #FOK
#     instCpTd0311.SetInputValue(8,'01') #01보통 03시장가 13최우선지정가
#     instCpTd0311.BlockRequest()
 
# #계좌 수익률 반환 함수
# def benefit_ratio():  # 계좌 수익률을 출력하는 함수 / 매도 감시 시 활용
 
#     instCpTd6033.SetInputValue(0,accountNumber)
#     instCpTd6033.SetInputValue(1,accFlag[0])
#     instCpTd6033.SetInputValue(3,'1')
#     instCpTd6033.BlockRequest()
 
#     #print(instCpTd6033.GetHeaderValue(8))  # 수익률 / 수익률을 기준으로 손절 익절 매도주문을 진행 / 수수료 감안해야함
#     ratio = instCpTd6033.GetHeaderValue(8)
#     return ratio
 
# #호가 출력 및 반환 함수
# def save_hoga_data(savestock_code):  # 호가 정보를 출력하는 함수 / 매도 주문 시 단가로 활용/ 1차 호가주문으로 활용해보자
 
#     instStockMst.SetInputValue(0, savestock_code)
#     instStockMst.BlockRequest()
 
#     save_buy_hoga_1 = instStockMst.GetDataValue(1, 0)  # 매수 1차 호가 // 매수 주문 시 활용
#     save_buy_hoga_2 = instStockMst.GetDataValue(1, 1)
#     save_buy_hoga_3 = instStockMst.GetDataValue(1, 2)
#     save_sell_hoga_1 = instStockMst.GetDataValue(0, 0)  # 매도 1차 호가
#     save_sell_hoga_2 = instStockMst.GetDataValue(0, 1)
#     save_sell_hoga_3 = instStockMst.GetDataValue(0, 2)
 
#     print('호가정보입니다요.')
#     print(instStockMst.GetDataValue(0,2))
#     print(instStockMst.GetDataValue(0,1))
#     print(instStockMst.GetDataValue(0,0))
#     print('--------------------')
#     print(instStockMst.GetDataValue(1,0))
#     print(instStockMst.GetDataValue(1,1))
#     print(instStockMst.GetDataValue(1,2))
 
#     return (save_buy_hoga_1,save_buy_hoga_2,save_buy_hoga_3,
#             save_sell_hoga_1,save_sell_hoga_2,save_sell_hoga_3)
 
# # 매도감시 함수 / 수익률이 익절선, 손절선 만족 시 매도 주문 실행 / 장중에만 실행시키려면?
# def sell_obsever():
#     print('매도 감시중입니다요. 주인님')
#     check = 0
#     #ratio_count = 0  # 임시로 무한루프를 종료하기 위한 기능
#     while check == 0:
#         time.sleep(1)
#         ratio = benefit_ratio()
#         print('현재수익률입니다요:',ratio)
#         ratio_count += 1
#         if ratio_count == 10:
#            print('종료처리합니다요')
#            check = 1
#            return 1
#         if ratio > 103: # 익절
#             print('얏호 익절 신호입니다요')
#             check = 1
#             return 1
#         elif ratio < 97: # 손절
#             print('손절입니다 도망치십시요~!')
#             check = 1
#             return 0
 
 
# #매도주문 함수
# def sell_order(item,quant,hoga):  # 매도주문
#     instCpTd0311.SetInputValue(0,'1') # 매도
#     instCpTd0311.SetInputValue(1,accountNumber)
#     instCpTd0311.SetInputValue(2,accFlag[0])
#     instCpTd0311.SetInputValue(3,item)
#     instCpTd0311.SetInputValue(4,quant)
#     instCpTd0311.SetInputValue(5,hoga)
#     instCpTd0311.SetInputValue(7,'2')
#     instCpTd0311.SetInputValue(8,'01')
#     instCpTd0311.BlockRequest()
 
 
 
# #프로그램 실행
 
# #연결 여부 체크
# connect_check()
 
# #타겟 리스트 셋팅
# now = int(datetime.now().strftime('%H%M'))
# if now < 810:
#     target_list_save_txt()
 
# target_list = target_list_read_txt()  # target_list 호출 //
 
# now = int(datetime.now().strftime('%H%M'))
 

# while now < 901 or now > 1530:
#     now = int(datetime.now().strftime('%H%M'))
#     print('장 중이 아닙니다요.')
#     time.sleep(15)
 
 
# while 1:
#     boughtcheck = bought_check()  # 0:미보유중 1:보유중
#     #보유 종목이 있을 경우 매도감시 및 매도주문 / 매도주문 완료 시 프로그램 종료
#     if boughtcheck == 1:
#         print('보유종목 정보를 가져와보겠습니다요')
#         savestock_name,savestock_quant,savestock_code = bought_item()
#         print(savestock_name,'종목을',savestock_quant,'주 보유중입니다요')
#         #장중일 경우 매도 감시/ 장중인 것은 어떻게 처리할 것인가
#         sell_check = sell_obsever()
#         if sell_check == 1 : #익절 신호 확인 시 매도 정보 추출
#             print('익절 신호입니다요!!') # 매수 1호가에 익절
#             save_buy_hoga_1, save_buy_hoga_2, save_buy_hoga_3,\
#             save_sell_hoga_1, save_sell_hoga_2, save_sell_hoga_3\
#             = save_hoga_data(savestock_code)
#             print(savestock_name,'종목',savestock_quant,'주를',save_buy_hoga_1,
#                   '원에 매도 주문 신청합니다요')
#             instCpTdUtil.TradeInit()
#             sell_order(savestock_code,savestock_quant,save_buy_hoga_1)
#             print('매도 주문 완료입니다 주인님')
#             print('먹은 돈 가지고 튑니다. 프로그램을 종료합니다')
#             quit()
 
#         elif sell_check == 0: # 손절 신호 확인 시 매도 정보 추출
#             print('손실입니다요!!')
#             save_buy_hoga_1, save_buy_hoga_2, save_buy_hoga_3, \
#             save_sell_hoga_1, save_sell_hoga_2, save_sell_hoga_3 \
#             = save_hoga_data(savestock_code)
#             print(savestock_name, '종목', savestock_quant, '주를', save_buy_hoga_1,
#                   '원에 매도 주문 신청합니다요')
#             instCpTdUtil.TradeInit()
#             sell_order(savestock_code,savestock_quant,save_buy_hoga_1)
#             print('매도 주문 완료입니다요 주인님')
#             print('오늘은 잃었습니다요 ㅜ')
#             print('내일을 기약하며 프로그램 종료합니다요')
#             quit()
 
#     if boughtcheck == 0 : # 보유종목이 없을 경우 장 전이라면 매수 종목 탐색
#         will_buy_item = buy_observer(instStockChart,target_list)
#         will_buy_item_name = instCpStockCode.CodeToName(will_buy_item[0])
#         item_buy_hoga_1, item_buy_hoga_2, item_buy_hoga_3,\
#         item_sell_hoga_1, item_sell_hoga_2, item_sell_hoga_3\
#         =item_hoga_data(will_buy_item[0])
#         possible_quant,possible_amount = buy_possible(will_buy_item[0],item_sell_hoga_1)
 
#         if possible_quant == 0:  #will_buy_item을 리스트로 활용할지 한 개의 종목만 담을지 //
#             print('총알이 부족합니다요,')
#             print('다른 종목을 찾습니다요')
#             target_list.remove(will_buy_item[0])
#             if len(target_list) == 0:
#                 print('살 수 있는 종모기 없읍니다요 ㅜㅜ')
#                 quit()
 
#             continue
 
#         if possible_quant > 0:
#             print('매수를 진행합니다요')
#             print(will_buy_item_name,'종목',possible_quant,'주를',
#                   item_sell_hoga_1,'원에 매수합니다요')
#             buy_order(will_buy_item[0],possible_quant,item_sell_hoga_1)
#             print('매수주문을 완료하였습니다요')
 
#             print('체결 완료 대기중입니다요')
#             buy_complete_check = 0
#             while buy_complete_check == 0 :
#                 savestock_name, savestock_quant, savestock_code\
#                 =bought_item()
#                 time.sleep(1)
#                 if possible_quant == savestock_quant:
#                     buy_complete_check = 1
#                     print('매수 체결 완료입니다요.')
 
#                     continue

